# Google Driveをマウント

In [0]:
from google.colab import drive
drive.mount('/content/drive')

!ls drive/My\ Drive

# 作業ディレクトリに移動

In [0]:
import os
!ls drive
print(os.getcwd())
os.chdir("drive/My Drive/data/fcn_keras")
print(os.getcwd())

# SetUp

In [0]:
% matplotlib inline
import os, re, imghdr
import keras
from keras import models
from keras.layers import Conv2D, MaxPooling2D, Flatten, Activation, Dense, Dropout, BatchNormalization
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from keras import backend as K

In [0]:
!ls
!ls models

In [0]:
from dataloader import DataLoader, Dataset
from models import *
from list_util import list_from_dir
from color import make_cmap
from transforms import label_to_img

N_CLASS = 21
INPUT_SIZE = (224,224)
VOC_CLASSES = [
    "background",
    "aeroplane",
    "bicycle",
    "bird",
    "boad",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "dining table",
    "dog",
    "horse",
    "motor_bike",
    "person",
    "potted_plant",
    "sheep",
    "sofa",
    "train",
    "tv"]

### ディレクトリ設定

In [0]:
log_dir = "log"
os.makedirs(log_dir, exist_ok=True)

train_data_dir = "../datasets/voc_semseg/train"
val_data_dir = "../datasets/voc_semseg//val"

train_img_dir = os.path.join(train_data_dir, 'img')
train_gt_dir = os.path.join(train_data_dir, 'gt')
val_img_dir = os.path.join(val_data_dir, 'img')
val_gt_dir = os.path.join(val_data_dir, 'gt')

#--------------------------------------
# DataLoader
#--------------------------------------
trn_dataset = Dataset(classes=N_CLASS, input_size=INPUT_SIZE,
                                        img_dir=train_img_dir, label_dir=train_gt_dir,
                                        trans=True)
val_dataset = Dataset(classes=N_CLASS, input_size=INPUT_SIZE,
                                        img_dir=val_img_dir, label_dir=val_gt_dir,
                                        trans=False)



train_loader = DataLoader(trn_dataset, batch_size=24, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=24, shuffle=False)

### データパス取得

In [0]:
train_data_paths = list_from_dir(train_img_dir, ('.jpg', '.png'))
train_gt_paths = list_from_dir(train_gt_dir, ('.jpg', '.png'))
val_data_paths = list_from_dir(val_img_dir, ('.jpg', '.png'))
val_gt_paths = list_from_dir(val_gt_dir, ('.jpg', '.png'))

print(len(train_data_paths))
print(len(train_gt_paths))
print(len(val_data_paths))
print(len(val_gt_paths))

# 学習

### Select model

In [0]:
# model = models.vgg_fcn8s.build(bilinear=True, drop_rate=0.5, weight_decay=0.00005)
model = models.vgg_fcn8s.build(bilinear=True, drop_rate=0, weight_decay=0)
# model = models.vgg_fcn32s.build(bilinear=True, drop_rate=0.5, weight_decay=0.00005)
# model = models.vgg_fcn32s.build(bilinear=True, drop_rate=0, weight_decay=0)
#  model = models.vgg_fcn16s.build(bilinear=True, drop_rate=0, weight_decay=0)
# model = models.vgg_fcn16s_ex.build(bilinear=True, drop_rate=0, weight_decay=0)
# model = models.vgg_fcn8s_ex.build(bilinear=True, drop_rate=0, weight_decay=0)
model.summary()

### Compile

In [0]:
w = os.path.join(log_dir,"fcn16s_bi_trans_classweight_weights-24-0.08-0.77-0.58-0.68-.hdf5")
model.load_weights(w, by_name=True)

# model = keras.models.load_model(os.path.join(log_dir, "fcn8.hdf5"))

In [0]:
class_weight = [
    1.,
    51.,
    151.,
    43.,
    64.,
    83.,
    23.,
    31.,
    15.,
    47.,
    51.,
    47.,
    25.,
    47.,
    36.,
    10.,
    86.,
    49.,
    46.,
    27.,
    51.
]

class_weight = np.ones((N_CLASS)) * class_weight
class_weight /= class_weight.sum()

print(class_weight)

class_weight_map = class_weight

In [0]:
def weighted_pixelwise_crossentropy(n_class, class_weights):

    def loss_func(y_true, y_pred):
        _, h, w, _ = K.int_shape(y_pred)
        class_weights_tensor = tf.convert_to_tensor(class_weights, dtype=tf.float32)
        
        flat_y_true  = tf.reshape(y_true, [-1, n_class])
        flat_y_pred = tf.reshape(y_pred, [-1, n_class])  
        
        
        epsilon =K.epsilon()
        flat_y_pred = tf.clip_by_value(flat_y_pred, epsilon, 1.)
        
        x_entorpy = tf.multiply(flat_y_true, tf.log(flat_y_pred))        
        weighted_x_entropy = tf.multiply(x_entorpy, class_weights_tensor)    
        weighted_x_entropy_mean = - tf.reduce_sum(weighted_x_entropy) / (h*w)

        return weighted_x_entropy_mean

    return loss_func

In [0]:
#--------------------------------------
# Optimzier
#--------------------------------------
optimizer = optimizers.SGD(lr=0.00001, momentum=0.9, nesterov=True)
#optimizer = optimizers.Adam()


#--------------------------------------
# Compile
#--------------------------------------
# model.compile(loss="categorical_crossentropy",
#                             optimizer=optimizer,
#                             metrics=["accuracy"])
model.compile(loss=weighted_pixelwise_crossentropy(N_CLASS, class_weight_map),
                            optimizer=optimizer,
                            metrics=["accuracy"])

#--------------------------------------
# Callback
#--------------------------------------
ckpt_name = 'weights-{epoch:02d}-{loss:.2f}-{acc:.2f}-{val_loss:.2f}-{val_acc:.2f}-.hdf5'
cbs = [
    ModelCheckpoint(os.path.join(log_dir, ckpt_name),
                    monitor='val_acc', verbose=0,
                    save_best_only=True,
                    save_weights_only=True,
                    mode='auto', period=1)
]

### Training

In [0]:
BATCH_SIZE = 24
EPOCHS =10

steps_per_epoch =  np.ceil(len(trn_dataset) / BATCH_SIZE)
validation_steps =  np.ceil(len(val_dataset) / BATCH_SIZE)

print("epochs : ", EPOCHS)
print("batch_size : ", BATCH_SIZE)
print("steps_per_epoch : ", steps_per_epoch)
print("validation_steps : ", validation_steps)

history = model.fit_generator(
    train_loader.flow(),
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=val_loader.flow(),
    validation_steps=validation_steps,
    callbacks=cbs,
    verbose=1)

### Save weights

In [0]:
weight_path = os.path.join(log_dir, "fcn8_ex.hdf5")
print(weight_path)
model.save(weight_path)
!ls log

In [0]:
weight_path = os.path.join(log_dir, "fcn8_weights.hdf5")
print(weight_path)
model.save_weights(weight_path)
!ls log

### Training accuracy

In [0]:
new_history=history

new_acc = new_history.history['acc']
new_val_acc = new_history.history['val_acc']
acc.extend(new_acc)
val_acc.extend(new_val_acc)

new_loss = new_history.history['loss']
new_val_loss = new_history.history['val_loss']
loss.extend(new_loss)
val_loss.extend(new_val_loss)

In [0]:
# acc = history.history['acc']
# val_acc = history.history['val_acc']


epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'b', label='train accuracy')
plt.plot(epochs, val_acc, 'r', label='val accuracy')
plt.xticks(np.arange(0,  len(acc)+1, step=10))
plt.legend()

plt.show()

### Training loss

In [0]:
# loss = history.history['loss']
# val_loss = history.history['val_loss']


epochs = range(1, len(acc)+1)

plt.plot(epochs, loss, 'b', label='train loss')
plt.plot(epochs, val_loss, 'r', label='val loss')
plt.xticks(np.arange(0,  len(acc)+1, step=10))

plt.legend()
plt.show()

# 予測

In [0]:
model = models.vgg_fcn16s.build(bilinear=True, drop_rate=0, weight_decay=0)
weight_path = os.path.join(log_dir, "fcn16s_bi_trans_weights-04-0.26-0.92-0.83-0.78-.hdf5")
print('load weight : ', weight_path)
model.load_weights(weight_path, by_name=True)

/root/.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
load weight :  log/fcn16s_bi_trans_weights-04-0.26-0.92-0.83-0.78-.hdf5


# eval

### Load weights

In [0]:
model = models.vgg_fcn32s.build(bilinear=True, drop_rate=0, weight_decay=0)

# w = os.path.join(log_dir,"fcn32s_bi_trans_weights-10-0.24-0.91-0.65-0.80-.hdf5")
w = os.path.join(log_dir,"fcn32s_bi_trans_class_weights-09-0.32-0.89-0.65-0.79-.hdf5")

model.load_weights(w, by_name=True)

optimizer = optimizers.SGD(lr=0.00001, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy",
                            optimizer=optimizer,
                            metrics=["accuracy"])

### Prediction

In [0]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [0]:
img_idx = 2

#---------------------------
# Load test image
#---------------------------
# test_dataset = Dataset(classes=N_CLASS, input_size=INPUT_SIZE,
#                                         img_dir=val_img_dir,
#                                         label_dir=val_gt_dir,
#                                         trans=False)
test_dataset = Dataset(classes=N_CLASS, input_size=INPUT_SIZE,
                                        img_dir=train_img_dir,
                                        label_dir=train_gt_dir,
                                        trans=False)
input_img, gt_img = test_dataset[img_idx]

#---------------------------
# Output prediction
#---------------------------
import time
start_time = time.time()
pred = model.predict(input_img)
end_time = time.time()
print("{} msec".format((end_time-start_time) * 1000))

loss, acc = model.evaluate(input_img, gt_img)
print("acc  : {}".format(acc*100))
print("loss : {}".format(loss*100))

print(pred.shape)


#---------------------------
# Show original image
#---------------------------
# im = Image.open(input_img_path)
# im = np.asarray(im)
# plt.imshow(im)
# plt.show()

#---------------------------
# Input img
#---------------------------
input_img_ = (input_img[0] + 1) * 127.5
input_img_ = np.uint8(input_img_)

#---------------------------
# GT img
#---------------------------
gt_img_ = label_to_img(gt_img[0])

#---------------------------
# Prediction img
#---------------------------
predicted_img = label_to_img(pred[0])

#---------------------------
# Show imgs
#---------------------------
plt.figure(figsize=(15,15))
img_list = [input_img_, gt_img_, predicted_img]
titel_list = ["input img", "gt img", "predicted img"]
plot_num = 1
for title, img in zip(titel_list, img_list):
    plt.subplot(1,3, plot_num)
    plt.title(title)
    plt.axis("off")
    plt.imshow(img)
    plot_num += 1

plt.show()

### クラスごとのaccを調査

In [0]:
def acc_per_class(pred, gt):
    # pred.shape is (HxWxC) and pred_idx.shape is (HxW)
    pred_idx = np.argmax(pred, axis=2)

    pred_maps = np.zeros(pred.shape)
    pred_h, pred_w, pred_ch = pred.shape

    for row in range(pred_h):
        for col in range(pred_w):
            idx = pred_idx[row, col]
            pred_maps[row, col, idx] = 1

    ans = pred_maps * gt

    for class_idx in range(pred_ch):
        gt_sum = gt[:,:,class_idx].sum()
        ans_sum = ans[:,:,class_idx].sum()

        if gt_sum != 0:
            acc = ans_sum / gt_sum * 100
        else:
            acc = 0
        print("class : {:02d}  gt_pixel : {}  ans_pixel : {}  acc : {}".format(class_idx, gt_sum, ans_sum, acc))
        
        
acc_per_class(pred[0], gt_img[0])


### クラスごとのピクセル数を調査

In [0]:
from PIL import Image


dataset = trn_dataset

input_img, gt_img = dataset[6]

print("---------------------------")
print("class pixel cnt")
for i, class_name in enumerate(VOC_CLASSES):
    class_cnt = np.count_nonzero(gt_img[0, :, :, i] == 1)
    print("{:02d} {} {}".format(i, class_name.ljust(15), class_cnt))
print("---------------------------")
    
plt.figure(figsize=(10,10))
plt.imshow(label_to_img(gt_img[0]))
plt.axis("off")
plt.show()

### データセット内の各クラスのピクセル数をカウント

In [0]:
def voc_count_class(dataset, n_class):
    voc_counter = np.zeros(n_class)
    for _, gt_img in dataset:
        for i in range(n_class):
            cnt = np.count_nonzero(gt_img[0, :, :, i] == 1)
            if cnt != 0:
                voc_counter[i] += int(cnt)
    return voc_counter

# GTを224x224でクロップしたときの各クラスのピクセル数をカウント
voc_counter = voc_count_class(dataset, N_CLASS)

In [0]:
print(voc_counter.sum())
print(voc_counter.sum() / (224*224))

### classごとの比率を計算

In [0]:
voc_class_sum = voc_counter.sum()
for i, class_cnt in enumerate(voc_counter):
    rate = class_cnt/voc_class_sum * 100
    print("{:.2f}% \t{:10} \t{} ".format(rate, int(class_cnt), VOC_CLASSES[i]))

### classごとのウエイトを計算

In [0]:
# class_weightを計算
voc_class_max = voc_counter.max()
for class_name, class_cnt in zip(VOC_CLASSES, voc_counter):
     print("{:4d}    \t{}".format(int(voc_class_max/class_cnt), class_name))